# Investment timelines - INDIGO Social Investment / Fund Manager Prototypes
The sample data used in this visualisation relate to four impact bonds in which BII is an investor: Cornwall Frequent Attenders, Street Impact Brighton, Pyramid Project and Skill Mill.

This visualization represents the flow of capital in a contract between the investor and provider in USD over the life of the contract including investment and repayment. 

## INDIGO database API endpoint
Setup INDIGO database API endpoint and helper methods for getting individual items from the API. This can be used with the `project`, `fund`, `organisation` and `assessment_resource` endpoints.

In [ ]:
import requests

import plotly.graph_objects as go

from datetime import datetime


INDIGO_DATABASE_API = 'https://golab-indigo-data-store.herokuapp.com/app/api1'


def api_get_item(endpoint, public_id):
    """
    Get individual item details from the API

    E.g. 
    item = api_get_item('project', 'INDIGO-POJ-0158')
    """
    try:
        response = requests.get(f'{INDIGO_DATABASE_API}/{endpoint}/{public_id}')
        item = response.json()
        return item
    except Exception as e:
        print(f'\nFailed to retrieve {endpoint} "{public_id}".\nError: {e}')
        return False





## Helper methods
Setup a method to simplify extracting values from the nested data dictionary. We also setup various helper methods for generating plot data for the visualisation. Most of these methods deal with calculating project start/end dates and timestamps.

In [ ]:
ACCEPTED_DATE_FORMATS = ['%Y-%m-%d', '%Y-%m', '%Y']


def extract_value(data, keys, default):
    """
    Safe method to get value from nested dictionary with default value fallback
    """
    try:
        result = data
        for key in keys:
            if result == default:
                break
            result = result.get(key) or default
        return result

    except Exception as e:
        print(f'Error: {e}')
        return default


def _get_current_date_unix():
    return datetime.now().timestamp()


def _convert_date_string_to_timestamp(time_string):
    for format in ACCEPTED_DATE_FORMATS:
        try:
            return datetime.strptime(time_string, format).timestamp()
        except ValueError:
            pass


def _turn_date_to_number_of_days(project_start_unix, string_value):
    if not string_value:
        return None
    value = _convert_date_string_to_timestamp(string_value)
    return int((value - project_start_unix) / (24*60*60))


def _get_project_start_end_dates(project_data):

    fields_start = [
        ['dates', 'outcomes_contract_signed', 'value'],
        ['dates', 'start_date_of_service_provision', 'value'],
    ]

    for field in fields_start:
        project_start = extract_value(project_data, field, '')
        if isinstance(project_start, str) and project_start.strip():
            project_start = project_start.strip()[0:7]
            project_start_unix = _convert_date_string_to_timestamp(project_start)
            break

    if not project_start:
        raise Exception("Project Start date is missing")

    fields_end = [
        ['dates', 'actual_completion_date', 'value'],
        ['dates', 'anticipated_completion_date', 'value'],
        ['dates', 'actual_end_date_of_service_provision', 'value'],
        ['dates', 'anticipated_end_date_of_service_provision', 'value'],
    ]

    project_end_unix = None
    project_end_days = None
    for field in fields_end:
        project_end = extract_value(project_data, field, '')
        if isinstance(project_end, str) and project_end.strip():
            project_end = project_end.strip()[0:7]
            project_end_unix = _convert_date_string_to_timestamp(project_end)
            project_end_days = _turn_date_to_number_of_days(project_start_unix=project_start_unix, string_value=project_end)
            break

    return project_start_unix, project_end_unix, project_end_days


def _get_number_of_days_to_now(project_start_unix):
    current_date_unix = _get_current_date_unix()
    return int((current_date_unix - project_start_unix) / (24*60*60))


def _get_final_points_to_add(values_to_plot, project_start_unix, project_end_unix, project_end_days):
    """
    This will generate a line from the final value to the end of the project or current date
    """
    current_date_unix = _get_current_date_unix()
    if project_end_unix and project_end_unix > current_date_unix:
        return [
            {
                'date': _get_number_of_days_to_now(project_start_unix),
                'value': values_to_plot[len(values_to_plot) - 1]['value'],
                'date_mode': 'now',
            },
            {
                'date': project_end_days,
                'value': values_to_plot[len(values_to_plot) - 1]['value'],
                'date_mode': 'future_end',
            }
        ]
    elif project_end_unix:
        return[{
            'date': project_end_days,
            'value': values_to_plot[len(values_to_plot) - 1]['value'],
            'date_mode': 'past_end',
        }]
    else:
        return[{
            'date': _get_number_of_days_to_now(project_start_unix),
            'value': values_to_plot[len(values_to_plot) - 1]['value'],
            'date_mode': 'no_end',
        }]


def _accumulate_values_across_data(data):
    accumulated_total = 0
    out = []
    for transaction_detail in data:
        accumulated_total += transaction_detail['value']
        out.append({
            'date': transaction_detail['date'],
            'value': accumulated_total
        })
    return out


def _get_transactions_for_viz_2_and_3(project_data, project_start_unix):
    # Get Transactions
    transaction_types_allowed = [
        'Interest Payment', 'Capital Repayment', 'Capital Repayment and Interest', 'Other'
    ]
    transactions = extract_value(project_data, ['transactions'], [])
    transactions = [
        transaction for transaction in transactions if
        extract_value(transaction, ['transaction_type', 'value'], None) in transaction_types_allowed and
        extract_value(transaction, ['investment_id', 'value'], None)
    ]

    # Get Investments
    transactions_minus = extract_value(project_data, ['transactions'], [])
    transactions_minus = [
        transaction for transaction in transactions_minus if
        extract_value(transaction, ['transaction_type', 'value'], None) == 'Debt' and
        extract_value(transaction, ['investment_id', 'value'], None)
    ]

    # Get all transaction details
    transaction_details_all = [
          {
              'date': _turn_date_to_number_of_days(
                  project_start_unix=project_start_unix,
                  string_value=extract_value(transaction, ['date', 'value'], None)
              ),
              'value': float(extract_value(transaction, ['money', 'amount_usd', 'value'], '0')),
          }
          for transaction in transactions
      ] + [
          {
              'date': _turn_date_to_number_of_days(
                  project_start_unix=project_start_unix,
                  string_value=extract_value(transaction, ['date', 'value'], None)
              ),
              'value': 0 - float(extract_value(transaction, ['money', 'amount_usd', 'value'], '0')),
          }
          for transaction in transactions_minus
      ]

    # Only get ones that have actual values we care about
    transaction_details = [
        transaction_detail for transaction_detail in transaction_details_all if
        transaction_detail['date'] is not None and transaction_detail['value']
    ]

    return transaction_details


def _get_min_date_or_zero_from_values_to_plot(values_to_plot):
    min_date = 0
    for v in values_to_plot:
        min_date = min(min_date, v['date'])
    return min_date

## Visualisation data
This is the main method for generating the plot data. It is called for each project.

In [ ]:
def visualisation_data_social_investment_prototype_2(project_data):
    try:

        # X axis - Time
        project_start_unix, project_end_unix, project_end_days = _get_project_start_end_dates(project_data)

        values_to_plot = _get_transactions_for_viz_2_and_3(project_data, project_start_unix)

        # Any data?
        if len(values_to_plot) == 0:
            return {'fail': 'No transactions of right type with an investment ID'}

        # this will start the line at 0
        values_to_plot.insert(0, {'date':  _get_min_date_or_zero_from_values_to_plot(values_to_plot), 'value': 0})
        # sort by date position
        values_to_plot = sorted(values_to_plot, key=lambda i: i['date'])
        # apply the accumulator for the transactions
        values_to_plot = _accumulate_values_across_data(values_to_plot)

        # this will generate a line from the final payment to the end of the project or current date
        values_to_plot.extend(_get_final_points_to_add(
            values_to_plot,
            project_start_unix,
            project_end_unix,
            project_end_days,
        ))

        return {'data': values_to_plot}

    except Exception as e:
        print('Error in visualisation_data_social_investment_prototype_2:' + str(e))
        return {'fail': str(e)}

## Plot helper method
Setup the plot build helper method that can create a `Plotly` scatter trace from the prototype data for a project.

In [ ]:
DATE_MODE_TO_MARKER_LOOKUP_FOR_VIZ_1_2_AND_3 = {
    'past_end': 'square',
    'future_end': 'triangle-left',
    'no_end': 'hourglass',
    'now': 'x',
    'unknown': 0,
}

MARKER_LINE_SIZE_SPECIAL = 20
MARKER_LINE_SIZE_NORMAL = 5


def build_plot(name, prototype):
    """
    Build a line/scatter plot
    """
    x = []
    y = []
    marker_symbol = []
    marker_size = []

    for detail in prototype.get('data'):
        x.append(detail['date'])
        y.append(detail['value'])

        marker_symbol.append(
            DATE_MODE_TO_MARKER_LOOKUP_FOR_VIZ_1_2_AND_3[detail.get('date_mode', 'unknown')]
        )

        marker_size.append(
            MARKER_LINE_SIZE_NORMAL if detail.get('date_mode', 'unknown') == 'unknown' else MARKER_LINE_SIZE_SPECIAL
        )

    return go.Scatter(
        name=name,
        x=x,
        y=y,
        marker_symbol=marker_symbol,
        marker_size=marker_size,
    )

## Get project data
Now we actually call the INDIGO API 'project' endpoint and retrieve the data for each of the projects of interest.

In [ ]:
# Call the API and pull down the data for each project
# and store in a dictionary for use later.
public_ids = ['INDIGO-POJ-0158', 'INDIGO-POJ-0167', 'INDIGO-POJ-0194', 'INDIGO-POJ-0195']

data = {}
endpoint = 'project'

for public_id in public_ids:
    data[public_id] = api_get_item(endpoint, public_id)

## Generate plot data
Pass each project to the `visualisation_data_social_investment_prototype_2` method and build a plot with the return values if they are valid.

In [ ]:
# Calculate the prototype data for each
plot_data = []

for public_id in public_ids:
    project_data = data[public_id]['project']['data']
    prototype = visualisation_data_social_investment_prototype_2(project_data)

    # Print out the return prototype data structure. It is possible that not
    # all projects have data available to plot.
    print(public_id, prototype)

    if not prototype or not prototype.get('data'):
       continue

    plot = build_plot(public_id, prototype)
    plot_data.append(plot)

## Build the figure

In [ ]:
fig = go.Figure(data=plot_data)

fig.update_traces(
    hovertemplate='%{x} days, %{y:.0f} USD',
)

fig.update_layout(
    height=800,
    title='Investment timelines',
    title_x=0.5,
    xaxis_title="Days since start",
    yaxis_title="USD",
    showlegend=True,
)

fig.show()

This visualization represents the flow of capital in a contract between the investor and provider in USD over the life of the contract including investment and repayment. 

All four projects are ongoing, and it is BII’s plan that the line in each project will eventually move up above zero USD – i.e. that there would be net positive return on investment.  

The x-axis represents time since the start of the project in days. The y-axis represents transaction amounts between investor and provider in USD. Negative values indicate a net flow of capital from the investor to the project provider. Positive values indicate flow of capital/returns paid to the investor. The data for this graph comes from the General Overview tab, where we take the “Date of Completion of the Service” data, and the “Transactions“ tab, where we take data related to investment and repayment. 

Therefore, this visualisation represents the behavior of the investment timeline in a live project (from the perspective of a social investor). At the beginning of the project, we can observe that the value of investment is negative: this means that the social investor has invested some resource (either debt or equity) and has not started to be repaid yet. As time goes by and outcomes are achieved, the investor starts to receive some return. We expect the line to go up every time the provider repays either capital or interest to the investor. This is usually aligned to when outcomes are expected to be achieved by the provider but outcomes payments are not shown directly on this visualization. 

 

In this version of the visualization, we show the flow of investment and repayment for four projects: Street Impact Brighton (INDIGO-POJ-0158), Cornwall Frequent Attenders (INDIGO-POJ-0167), Pyramid Project (INDIGO-POJ-0194) and The Skill Mill (INDIGO-POJ-0195). This visualization shows that for the Street Impact Brighton project, 2 ½ years into a 3-year project the investor has not yet been fully repaid.  


As this is a live project the expected J-curve over the investment timeline is incomplete. One limitation of this visualization is that it focuses on the financial return which does not show the full picture for a social investor who has to consider both risk and return of impact alongside the financial. For example, a project would not be considered ‘successful’ if the financial investment was returned above the initial value of capital invested but this was to the detriment of anticipated impact to be achieved.

## Important Notice and Disclaimer on INDIGO Data
<sub><sup>
INDIGO data are shared for research and policy analysis purposes. INDIGO data can be used to support a range of insights, for example, to understand the social outcomes that projects aim to improve, the network of organisations across projects, trends, scales, timelines and summary information. The collaborative system by which we collect, process, and share data is designed to advance data-sharing norms, harmonise data definitions and improve data use. These data are NOT shared for auditing, investment, or legal purposes. Please independently verify any data that you might use in decision making. We provide no guarantees or assurances as to the quality of these data. Data may be inaccurate, incomplete, inconsistent, and/or not current for various reasons: INDIGO is a collaborative and iterative initiative that mostly relies on projects all over the world volunteering to share their data. We have a system for processing information and try to attribute data to named sources, but we do not audit, cross-check, or verify all information provided to us. It takes time and resources to share data, which may not have been included in a project’s budget. Many of the projects are ongoing and timely updates may not be available. Different people may have different interpretations of data items and definitions. Even when data are high quality, interpretation or generalisation to different contexts may not be possible and/or requires additional information and/or expertise. Help us improve our data quality: email us at indigo@bsg.ox.ac.uk if you have data on new projects, changes or performance updates on current projects, clarifications or corrections on our data, and/or confidentiality or sensitivity notices. Please also give input via the INDIGO Data Definitions Improvement Tool and INDIGO Feedback Questionnaire.
</sup></sub>
